# 4장 - 컴프리헨션과 제너레이터


- 많은 프로그램이 리스트, 딕셔너리의 키/값 쌍, 집합 처리를 중심으로 만들어진다.
- 파이썬에서는 "컴프리헨션(comprehension) 이라는 특별한 구문을 사용해 이런 타입을 간결하게 이터레이션하면서 원소로부터 파생되는 데이터 구조를 생성할 수 있다.

- 컴프리헨션 코딩 스타일은 제너레이터(generator)를 사용하는 함수로 확장할 수 있다.
- 제너레이터는 함수가 점진적으로 반환하는 값으로 이뤄지는 스트림을 만들어준다.


- 이터레이터를 사용할 수 있는 곳(for loop, 별표 식 등)이라면 어디에서나 제니레이터 함수를 호출한 결과를 사용할 수 있다.
- 제너레이터를 사용하면 성능을 향상시키고, 메모리 사용을 줄이고, 가독성을 높일 수 있다.

### BETTER WAY 27 - map과 filter 대신 컴프리헨션을 사용하라

- 파이썬은 다른 시퀀스나 이터러블에서 새 리스트를 만들어내는 간결한 구문을 제공한다. -> 이런 식을 <font color='blue'>리스트 컴프리헨션</font> 이라고 한다.
- 딕셔너리에는 딕셔너리 컴프리헨션
- 집합에는 집합 컴프리헨션이 있다.

In [3]:
# 리스트 모든 원소의 제곱을 계산하는 코드
a = [1, 2, 3, 4, 5, 6, 7, 8]
squres = []
for x in a:
    squres.append(x**2)
print(squres)

[1, 4, 9, 16, 25, 36, 49, 64]


In [4]:
# 리스트 컴프리헨션을 사용
squares = [x**2 for x in a]
print(squares)

[1, 4, 9, 16, 25, 36, 49, 64]


In [5]:
# 짝수만 제곱하고 싶으면?
even_squares = [x**2 for x in a if x % 2 == 0]
print(even_squares)

[4, 16, 36, 64]


In [7]:
# filter 내장 함수를 map과 함께 사용해서도 같은 결과를 얻을 수 있지만, 이렇게 만든 코드는 읽기가 어렵다
alt = map(lambda x: x**2, filter(lambda x: x % 2 ==0, a))
print(list(alt))

[4, 16, 36, 64]


### BETTER WAY 28 - 컴프리헨션 내부에 제어 하위 식을 세 개 이상 사용하지 말라

- 컴프리헨션은 루프를 여러 수준으로 내포하도록 허용한다.

In [11]:
# 1단계 컴프리헨션
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
flat = [x for row in matrix for x in row]
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [12]:
# 2단계 컴프리헨션 - 여전히 읽기 쉬운 편이다
squared = [[x**2 for x in row] for row in matrix]
print(squared)

[[1, 4, 9], [16, 25, 36], [49, 64, 81]]


In [14]:
# 3단계 컴프리헨션
my_lists = [
    [[1, 2, 3], [4, 5, 6]],
    [[7, 8, 9], [10, 11, 12]],
    [[13, 14, 15], [16, 17, 18]],
]
flat = [x for sublist1 in my_lists
       for sublist2 in sublist1
       for x in sublist2]
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [16]:
# 3단계 정도가 되면, 일반 루프문을 사용해 같은 결과를 만들어내는 코드가 더 명확해 보인다.
flat = []
for sublist1 in my_lists:
    for sublist2 in sublist1:
        flat.extend(sublist2)
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


- 그러므로, 컴프리헨션에 들어가는 하위 식이 3개 이상 되지 않게 제한하라는 규칙을 지켜라

### BETTER WAY 29 - 대입식을 사용해 컴프리헨션 안에서 반복 작업을 피하라

- 컴프리헨션에서 같은 계산을 여러 위치에서 공유하는 경우가 흔하다

In [17]:
# 예) 주문관리 프로그램
# 고객의 요청이 재고 수량을 넘지 않고, 배송에 필요한 최소 수량(8개 이상)을 만족하는지 확인

stock = {
    '못': 125,
    '나사못': 35,
    '나비너트': 8,
    '와셔': 24
}

order = ['나사못', '나비너트', '클립']

def get_batches(count, size):
    return count // size

result = {}
for name in order:
    count = stock.get(name, 0)
    batches = get_batches(count, 8)
    if batches:
        result[name] = batches
        
print(result)

{'나사못': 4, '나비너트': 1}


In [18]:
# 이걸 딕셔너리 컴프리헨션을 사용하면 더 간결해진다.
found = {name: get_batches(stock.get(name, 0), 8)
        for name in order
        if get_batches(stock.get(name, 0), 8)}
print(found)

{'나사못': 4, '나비너트': 1}


In [20]:
# 하지만, 위 코드에서는
# stock.get(name, 0) 이게 반복적으로 호출된다.

# 쉬운 해법은 왈러스 연산자를 사용하는 것이다.
found = {name: batches
        for name in order
        if (batches := get_batches(stock.get(name, 0), 8))}
print(found)

{'나사못': 4, '나비너트': 1}


### BETTER WAY 30 - 리스트를 반환하기보다는 제너레이터를 사용하라

- 시퀀스를 결과로 만들어내는 함수를 만들 때 가장 간단한 선택은 원소들이 모인 리스트를 반환하는 것

In [23]:
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        #print(letter)
        if letter == ' ':
            result.append(index + 1)
    return result

address = '컴퓨터(영어: Computer, 문화어: 콤퓨터, 순화어: 전산기)는 진공관'
result = index_words(address)
print(result)

[0, 8, 18, 23, 28, 33, 39]


In [32]:
# 위 함수는 잡음이 많고 핵심을 알아보기 어렵다.. 개선해보자
# 개선방법은 제너레이터를 사용하는 것이다.
# 제너레이터는 yield 식을 사용하는 함수에 의해 만들어진다.

def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1
            
address = '컴퓨터(영어: Computer, 문화어: 콤퓨터, 순화어: 전산기)는 진공관'
it = index_words_iter(address)
#print(next(it))
#print(next(it))
#print(next(it))
#print(next(it))
#print(next(it))
#print(next(it))
#print(next(it))
print(list(it))

[0, 8, 18, 23, 28, 33, 39]


### BETTER WAY 31 - 인자에 대해 이터레이션할 때는 방어적이 돼라

- 객체가 원소로 들어 있는 리스트를 함수가 파라미터로 받았을 때, 이 리스트를 여러 번 이터레이션하는 것이 중요할 때가 종종 있다

In [56]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [50]:
def read_visits():
    items = [15, 35, 80]
    for x in items:
        yield int(x)

print(list(read_visits()))

[15, 35, 80]


In [51]:
# 놀랍게도 normalize 함수에 read_visits 가 반환한 값을 전달하면 아무 결과도 나오지 않는다.
it = read_visits()
percentages = normalize(it)
print(percentages)

[]


In [52]:
# 이유는?
# 이터레이터가 결과를 한 번만 만들어내기 때문이다
# 이미 StopIteration 예외가 발생한 이터레이터나 제너레이터를 다시 이터레이션하면 아무 결과도 얻을 수 없다.
it = read_visits() # 여기서 이미 iteration 이 끝났다.
print(list(it))
print(list(it)) # 한 번 더 it 를 사용하려고 하면 빈 값이다.

[15, 35, 80]
[]


In [53]:
# 1) 쉽게 해결하려면, 입력 데이터를 명시적으로 소진시키고 이터레이터의 전체 내용을 리스트에 넣으면 된다.
def normalize_copy(numbers):
    numbers_copy = list(numbers) # 이터레이터 복사
    total = sum(numbers_copy)
    result = []
    for value in numbers_copy:
        percent = 100 * value / total
        result.append(percent)
    return result

it = read_visits()
percentages = normalize_copy(it)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [54]:
# 잘 동작하는 것처럼 보이나, 이런 접근 방식의 문제점은 "입력 데이터의 내용을 복사하면 메모리를 엄청나게 사용할 수 있다" 는 것이다.
#
# 2) 호출할때마다 새로 이터레이터를 반환하는 함수를 받는 것
def normalize_func(get_iter):
    total = sum(get_iter()) # 새로운 이터레이터
    result = []
    for value in get_iter(): # 새로운 이터레이터
        percent = 100 * value / total
        result.append(percent)
    return result

percentages = normalize_func(lambda: read_visits()) # 람다 함수를 넘긴다
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [59]:
# 작동하기는 하지만, 이렇게 람다 함수를 넘기는 것은 보기에 좋지 않다.
#
# 3) 더 나은 방법은, 이터레이터 프로토콜(iterator protocol)을 구현한 새로운 컨테이너 클래스를 제공하는 것
#
# 정의한 클래스에 __iter__ 메서드를 제너레이터로 구현하기만 하면, 이 모든 동작을 만족시킬 수 있다.
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

class ReadVisits:
    def __iter__(self):
        items = [15, 35, 80]
        for x in items:
            yield int(x)

visits = ReadVisits()
percentages = normalize(visits)
print(percentages)
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]
[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [60]:
# 잘 동작한다.
# 이유는?
# normalize 함수 내 sum 메서드가 ReadVisits.__iter__ 를 호출해서 새로운 이터레이터 객체를 할당하기 때문이다.

### BETTER WAY 32 - 긴 리스트 컴프리헨션보다는 제너레이터 식을 사용하라

- map과 filter대신 컴프리헨션을 사용하라의 문제점은 입력 시퀀스와 같은 수의 원소가 들어 있는 리스트를 만들어낼 수 있다는 것이다.

In [65]:
value = [int(x) for x in open('my_file.txt')]
print(value)

[100, 57, 15, 1, 12, 75, 5, 86, 89, 11]


In [66]:
# 입력이 작으면 문제가 없지만, 입력이 커지면 메모리를 상당히 많이 사용하고, 그로 인해 프로그램이 중단될 수 있다.
# 이 문제를 해결하기 위해 파이썬은 제너레이터(generator expression) 식을 제공한다.
# [ ] 대신 ( ) 를 쓰면 된다.
it = (int(x) for x in open('my_file.txt'))
print(next(it))

100


In [67]:
print(next(it))
print(next(it))
print(next(it))
print(next(it))

57
15
1
12


### BETTER WAY 33 - yield from을 사용해 여러 제너레이터를 합성하라

In [69]:
def move(period, speed):
    for _ in range(period):
        yield speed
        
def pause(delay):
    for _ in range(delay):
        yield 0
        
def animate():
    for delta in move(4, 5.0):
        yield delta
    for delta in pause(3):
        yield delta
    for delta in move(2, 3.0):
        yield delta
        
def render(delta):
    print(f'Delta: {delta:.1f}')

def run(func):
    for delta in func():
        render(delta)

run(animate)

Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 0.0
Delta: 0.0
Delta: 0.0
Delta: 3.0
Delta: 3.0


In [72]:
# 이 코드의 문제점은?
#
# animate가 너무 반복적이라는 것이다. 
#
# 이 문제의 해법은 yield from 식을 사용하는 것이다. 
# 이는 고급 제너레이터 기능으로, 제어를 부모 제너레이터에게 전달하기 전에 내포된 제너레이터가 모든 값을 내보낸다.

def animate_composed():
    yield from move(4, 5.0)
    yield from pause(3)
    yield from move(2, 3.0)

run(animate_composed)

Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 0.0
Delta: 0.0
Delta: 0.0
Delta: 3.0
Delta: 3.0


### BETTER WAY 34 - send로 제너레이터에 데이터를 주입하지 말라

- yield 식을 사용하면 제너레이터 함수가 간단하게 이터레이션이 가능한 출력을 만들 수 있다.
- 하지만 이렇게 만들어내는 채널은 단방향이다.
- 제너레이터가 데이터를 내보내면서 다른 데이터를 받아들이는.. 이런 양방향 통신이 있다면 많은 경우에 도움이 될 것이다.

In [74]:
# 예) 소프트웨어 라디오를 사용해 신호를 내보낸다.

import math

def wave(amplitude, steps):
    step_size = 2 * math.pi / steps        # 2 라디안/단계 수
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        output = amplitude * fraction
        yield output

# 이제 wave 제너레이터를 이터레이션 하면서 진폭이 고정된 파형 신호를 송신할 수 있다.
def transmit(output):
    if output is None:
        print(f'출력: None')
    else:
        print(f'출력: {output:>5.1f}')
        
def run(it):
    for output in it:
        transmit(output)

run(wave(3.0, 8))

출력:   0.0
출력:   2.1
출력:   3.0
출력:   2.1
출력:   0.0
출력:  -2.1
출력:  -3.0
출력:  -2.1


In [77]:
# 기본 파형을 생성하는 한 이 코드는 잘 동작한다.
#
# 하지만, 별도의 입력을 사용해 진폭을 지속적으로 변경해야 한다면 이 코드는 쓸모가 없다.
#
'''
파이썬 제너레이터는 send 메서드를 지원한다. 이 메서드는 yield 식을 양방향 채널로 격상시켜준다.
send 메서드를 사용하면 입력을 제너레이터에 스트리밍하는 동시에 출력을 내보낼 수 있다.
일반적으로 제너레이터를 이터레이션 할 때 yield 식이 반환하는 값은 None 이다.
'''

def my_generator():
    received = yield 1
    print(f'받은 값: {received}')
    
it = iter(my_generator())
output = next(it)                 # 첫 번째 제너레이터 출력을 얻는다.
print(f'출력 값: {output}')

try:
    next(it)                       # 종료될 때까지 제너레이터를 실행한다.
except StopIteration:
    pass

출력 값: 1
받은 값: None


In [78]:
it = iter(my_generator())
output = it.send(None)           # 첫 번째 제너레이터 출력을 얻는다.
print(f'출력 값: {output}')

try:
    it.send('안녕!')              # 값을 제너레이터에 넣는다.
except StopIteration:
    pass

출력 값: 1
받은 값: 안녕!


In [80]:
# 이런 동작을 활용해 입력 시그널을 바탕으로 사인 파의 진폭을 변조할 수 있다.
# 먼저 yield 식이 반환한 진폭 값을 amplitude에 저장하고, 다음 yield 출력 시 이 진폭 값을 활용하도록 wave 제너레이터를 변경해야 한다.

def wave_modulating(steps):
    step_size = 2 * math.pi / steps        # 2 라디안/단계 수
    amplitude = yield                # 초기 진폭을 받는다
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        output = amplitude * fraction
        amplitude = yield output     # 다음 진폭을 받는다

def transmit(output):
    if output is None:
        print(f'출력: None')
    else:
        print(f'출력: {output:>5.1f}')
        
def run_modulating(it):
    amplitudes = [None, 7, 7, 7, 2, 2, 2, 2, 10, 10, 10, 10, 10]
    for amplitude in amplitudes:
        output = it.send(amplitude)
        transmit(output)
        
run_modulating(wave_modulating(12))

출력: None
출력:   0.0
출력:   3.5
출력:   6.1
출력:   2.0
출력:   1.7
출력:   1.0
출력:   0.0
출력:  -5.0
출력:  -8.7
출력: -10.0
출력:  -8.7
출력:  -5.0
